In [11]:
import itertools
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, fixed
from functools import partial

In [12]:
def build_kernel(kernel, **kwargs):
    return partial(kernel, **kwargs)

def get_gram_matrix(X, kernel):
    n = len(X)
    gram_matrix = np.array([np.zeros(n) for _ in range(n)])
    for i, j in itertools.product(range(n), range(n)):
        gram_matrix[i][j] = kernel(X[i], X[j])
    return gram_matrix

def get_mean_vector(X):
    return np.zeros(len(X))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [32]:
def kernel_simulator(X, kernel, classes=2, **hyperparams):
    mean_vector = get_mean_vector(X)
    cov = get_gram_matrix(X, build_kernel(kernel=kernel, **hyperparams))

    fig, ax = plt.subplots(figsize = (10, 10))
    colors = list(mcolors.TABLEAU_COLORS.keys())
    samples = np.array([sigmoid(np.random.multivariate_normal(mean = mean_vector, cov=cov)) for _ in range(classes)])
    class_labels = np.argmax(samples, axis=0)
    for i in range(classes):
        ax.plot(X, samples[i], color=colors[i])
    ax.scatter(X, np.ones_like(X), color=[colors[c] for c in class_labels])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_ylim(0,1)

In [34]:
def linear_kernel(x, y, variance, **kwargs):
    return variance * np.dot(x, y)

def gaussian_kernel(x, y, variance, length, **kwargs):
    return variance * np.exp(-(np.dot((x-y), (x-y))/(2*length)))

def periodic_kernel(x1, x2, amplitude, length, period, **kwargs):
    return amplitude **2 * np.exp(- 2/length**2 * np.sin(np.pi * np.abs(x1 - x2) / period)**2)

def wiener_kernel(x, y, variance, **kwargs):
    return variance * np.min(x, y)

def rational_quadratic_kernel(x, y, variance, length, scale, **kwargs):
    return variance * (1 + np.dot((x-y), (x-y)) ** 2 / (2 * scale * length **2)) ** (-scale)

def polynomial_kernel(x, y, c, deg, **kwargs):
    return (np.dot(x, y) + c) ** deg

X = np.arange(0., 10., 0.1)
kernels = [
    gaussian_kernel, 
    linear_kernel, 
    wiener_kernel, 
    periodic_kernel, 
    rational_quadratic_kernel,
    polynomial_kernel
]

In [35]:
interact(
    kernel_simulator, 
    X=fixed(X), 
    classes=(2,5,1),
    kernel=kernels, 
    variance=(1,10, 0.2),
    amplitude=(0.1, 10, 0.2),
    period=(0.2, 10, 0.2),
    length=(0.1, 10, 0.2),
    scale = (0., 10., 0.2),
    c=(1,5, 0.5),
    deg=(0,8, 0.2),
    replot=(0,1, 1)
)

interactive(children=(Dropdown(description='kernel', options=(<function gaussian_kernel at 0x7f6c48144ee0>, <f…

<function __main__.kernel_simulator(X, kernel, classes=2, **hyperparams)>